# Dont run 1st cell


In [7]:
a={}

In [16]:
from PIL import Image
import cv2
import os
import numpy as np
from sklearn.svm import SVC
import matplotlib.pyplot as plt
print("Enter your ID and then your name")

Enter your ID and then your name


In [10]:
list=[int(input()),input()]

2
papa


In [11]:
a[list[0]]=list[1]



In [12]:
a

{1: 'harshit', 2: 'papa'}

In [ ]:
# creating a dataset folder with images
def assure_path_exists(path):
    dir = os.path.dirname(path)
    if not os.path.exists(dir):
        os.makedirs(dir)
# Start capturing video 
vid_cam = cv2.VideoCapture(0)
# Detect object in video stream using Haarcascade Frontal Face
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# For each person, one face id
face_id = list[0]
# Initialize sample face image
count = 0
assure_path_exists("dataset/")
# Start looping
while(True):
    # Capture video frame
    _, image_frame = vid_cam.read()
    # Convert frame to grayscale
    gray = cv2.cvtColor(image_frame, cv2.COLOR_BGR2GRAY)
    # Detect frames of different sizes, list of faces rectangles
    faces = face_detector.detectMultiScale(gray, 1.3, 5)
    # Loops for each faces
    for (x,y,w,h) in faces:
        cv2.rectangle(image_frame, (x,y), (x+w,y+h), (255,0,0), 2)
        count += 1
        # Save the captured image into the datasets folder
        cv2.imwrite("dataset/User." + str(face_id) + '.' + str(count) + ".jpg", gray[y:y+h,x:x+w])
        # Display the video frame, with bounded rectangle on the person's face
        cv2.imshow('frame', image_frame)
    # To stop taking video, press 'q' for at least 100ms
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break
    # If image taken reach 100, stop taking video
    elif count>100:
        break
# Stop video
vid_cam.release()
# Close all started windows
cv2.destroyAllWindows()


In [13]:
# Creating hog array
# Import Python Image Library (PIL)

hog=cv2.HOGDescriptor()
def assure_path_exists(path):
    dir = os.path.dirname(path)
    if not os.path.exists(dir):
        os.makedirs(dir)
def getImagesAndLabels(path):
    # Get all file path
    h = 320
    w = 160
    array=np.array([])
    ids=[]
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]  
    img = Image.open(imagePaths[0]).convert('L')
    img1 = np.array(img,'uint8')
    img=cv2.resize(img1, (w, h), interpolation=cv2.INTER_CUBIC)
    h1=hog.compute(img,winStride=(64,128),padding=(0,0))	#storing HOG features as column vector
    h_trans=h1.transpose()	#transposing the column vector
    array=np.vstack(h_trans)	#appending it to the array
    id = int(os.path.split(imagePaths[0])[-1].split(".")[1])
    ids.append(id)
    for i in range(1,len(imagePaths)):
        img = Image.open(imagePaths[i]).convert('L')
        img1 = np.array(img,'uint8')
        img=cv2.resize(img1, (w, h), interpolation=cv2.INTER_CUBIC)
        h1=hog.compute(img,winStride=(64,128),padding=(0,0))	#storing HOG features as column vector
        h_trans=h1.transpose()	#transposing the column vector
        array=np.append(array,h_trans,axis=0)	#appending it to the array
        id = int(os.path.split(imagePaths[i])[-1].split(".")[1])
        ids.append(id)	#appending it to the array
    return array,ids
hogarray,ids= getImagesAndLabels('dataset')
        
len(ids)

202

In [15]:
clf=SVC(gamma=0.001,C=10)
clf.fit(hogarray,ids)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [17]:
# Load prebuilt model for Frontal Face
cascadePath = "haarcascade_frontalface_default.xml"
# Create classifier from prebuilt model
faceCascade = cv2.CascadeClassifier(cascadePath);
# Set the font style
font = cv2.FONT_HERSHEY_SIMPLEX
#Initialize and start the video frame capture
scam = cv2.VideoCapture(0)
# Loop
print(scam.isOpened())
h1 = 320
w1 = 160
while (True):
    # Read the video frame
    ret, im =scam.read()
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    # Get all face from the video frame
    faces = faceCascade.detectMultiScale(gray, 1.3,5)
    for(x,y,w,h) in faces:
        # Create rectangle around the face
        cv2.rectangle(im, (x-20,y-20), (x+w+20,y+h+20), (0,255,0), 2)
        # Recognize the face belongs to which ID
        gray=gray[y:y+h,x:x+w]
        imgtest=cv2.resize(gray, (w1, h1), interpolation=cv2.INTER_CUBIC)
        hTest=hog.compute(imgtest,winStride=(64,128),padding=(0,0))
        h_transTest=hTest.transpose()
        arrayTest=np.vstack(h_transTest)
        Id= int(clf.predict(arrayTest))
        if(a.get(Id)!=None):
            Id = a.get(Id)
        cv2.rectangle(im, (x-22,y-90), (x+w+22, y-22), (0,255,0), -1)
        cv2.putText(im, str(Id), (x,y-40), font, 0.8, (255,255,255), 3)
        cv2.imshow('im',im) 
    # If 'q' is pressed, close program
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break
# Stop the camera
scam.release()
# Close all windows
cv2.destroyAllWindows()

True
